In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(1.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(1.5)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(1.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(1.5)

# click next page button
def click_next_page_button():
    time.sleep(1.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(1.5)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'lxml')
        print('page start')
        time.sleep(1)
        name = get_player_name()
        time.sleep(1)
        position = get_player_position()
        time.sleep(1)
        current_data = get_data()
        time.sleep(1)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    df = pd.DataFrame(df_list, columns=get_column_name())
    df.to_csv(f'mlb_stats/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
    print('This is the last page :)')
    print(df)

In [3]:
urls = pd.read_csv('mlb_stats_urls.csv', encoding='utf-8')

for year in range(2013, 2015):
    for link in urls['link']:
        url = link + str(year) + '?playerPool=ALL'
        driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
        html = driver.get(url)
        sp = soup(driver.page_source, 'lxml')

        try:
            df_list = []
            while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                click_next_page_button()
        except:
            print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=get_column_name())
        df.to_csv(f'mlb_stats/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)


C:\Users\user\AppData\Local\Temp\ipykernel_24812\2755073102.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
This is the last page :)
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0        John Hester        C  LAA    1    0    1    0   0  0   0  ...  -.--   
1         Mike Trout       CF  LAA  157  589  109  190  39  9  27  ...  0.83   
2       Kole Calhoun       RF  LAA   58  195   29   55   7  2   8  ...  1.04   
3     Howie Kendrick       2B  LAA  122  478   55  142  21  4  13  ...  1.66   
4      Albert Pujols       DH  LAA   99  391   49  101  19  0  17  ...  0.78   
5      Efren Navarro       1B  LAA    4    4    0    1   0  0   0  ...  2.00   
6        Mark Trumbo       1B  LAA  159  620   85  145  30  2  34  ...  1.17   
7      Josh Hamilton       RF  LAA  151  576   73  144  32  5  21  ...  0.81   
8     Chris Iannetta        C  LAA  115  325   40   73  15  0  11  ...  0.71   
9        Grant Green       2B  LAA   40  125   16   35   8  1   1  ...  0.83   
10       Hank Conger        C  LAA   92  233   23   58  13  1   7  ...  0

C:\Users\user\AppData\Local\Temp\ipykernel_24812\2755073102.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
This is the last page :)
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0        Jason Castro        C  HOU  120  435  63  120  35  1  18  ...  0.95   
1        Chris Carter       1B  HOU  148  506  64  113  24  2  29  ...  0.65   
2       Lucas Harrell        P  HOU   37    4   0    1   0  0   0  ...  1.00   
3         Rick Ankiel       RF  HOU   25   62   6   12   3  0   5  ...  0.36   
4       Brett Wallace       1B  HOU   79  262  35   58  14  1  13  ...  1.00   
5           L.J. Hoes       RF  HOU   46  167  24   48   7  2   1  ...  2.31   
6     Robbie Grossman       LF  HOU   63  257  29   69  14  0   4  ...  1.07   
7      Justin Maxwell       CF  HOU   40  137  21   33  10  2   2  ...  1.03   
8      Matt Dominguez       3B  HOU  152  543  56  131  25  0  21  ...  0.95   
9         Jose Altuve       2B  HOU  152  626  64  177  31  2   5  ...  1.24   
10        Carlos Pena       1B  HOU   85  277  38   58  13  1   8  ...  1

C:\Users\user\AppData\Local\Temp\ipykernel_24812\2755073102.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
This is the last page :)
              PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0        Kurt Suzuki        C  OAK   15   33   6   10   2  0   2  ...  0.43   
1     Josh Donaldson       3B  OAK  158  579  89  174  37  3  24  ...  1.04   
2       Brandon Moss       1B  OAK  145  446  73  114  23  3  30  ...  0.61   
3         Jed Lowrie       SS  OAK  154  603  80  175  45  2  15  ...  0.59   
4         Coco Crisp       CF  OAK  131  513  93  134  22  3  22  ...  0.86   
5   Alberto Callaspo       2B  OAK   50  159  20   43   7  0   5  ...  0.74   
6          John Jaso        C  OAK   70  207  31   56  12  0   3  ...  0.96   
7       Derek Norris        C  OAK   98  264  41   65  16  0   9  ...  0.66   
8    Yoenis Cespedes       LF  OAK  135  529  74  127  21  4  26  ...  0.63   
9       Daric Barton       1B  OAK   37  104  15   28   2  0   3  ...  0.76   
10        Seth Smith       DH  OAK  117  368  49   93  27  0   8  ...  1.06   
11   

C:\Users\user\AppData\Local\Temp\ipykernel_24812\2755073102.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
This is the last page :)
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0   Edwin Encarnacion       1B  TOR  142  530  90  144  29  1  36  ...  0.79   
1       Jose Bautista       RF  TOR  118  452  82  117  24  0  28  ...  0.85   
2           Adam Lind       1B  TOR  143  465  67  134  26  1  23  ...  1.37   
3        Colby Rasmus       CF  TOR  118  417  57  115  26  1  22  ...  0.65   
4         R.A. Dickey        P  TOR   34    6   2    2   1  0   0  ...  1.00   
5       Moises Sierra       RF  TOR   35  107  11   31  13  1   1  ...  1.00   
6          Jose Reyes       SS  TOR   93  382  58  113  20  0  10  ...  1.07   
7         Mark DeRosa        X  TOR   88  204  23   48  12  1   7  ...  1.29   
8        Brett Lawrie       3B  TOR  107  401  41  102  18  3  11  ...  0.97   
9        Anthony Gose       CF  TOR   52  147  15   38   6  5   2  ...  1.52   
10        Rajai Davis       LF  TOR  108  331  49   86  16  2   6  ...  0

C:\Users\user\AppData\Local\Temp\ipykernel_24812\2755073102.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start


In [ ]:
driver.quit()